In [1]:
import torch
import torchaudio
from nnAudio import features
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
import lightning.pytorch as pl
from lightning.pytorch.loggers import TensorBoardLogger
import os
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Audio
from IPython.display import clear_output, display
import librosa
import math
import datetime
import time
import numpy as np
import random

#Eigene Sachen
from train import *
from data.data import *
from data.data_other import *
from util.util import *
from util.helper import *
from util.datasets import *
from util.audio_processing import *
import util.plot as plot
import util.metric as metric
import models
import tqdm

In [2]:
from models.Simple import *
from models.Simple.SimpleLightningBase import *
from models.Simple.DeepCNN import *
from models.Simple.STFTCNN import *
from torchaudio.transforms import MelScale

In [3]:
#Model Params
SAMPLE_LENGTH  = 512
HOP_LENGTH     = 512
CONTEXT_LENGTH = 0

In [4]:
#Param
LENGTH_CHECK = 60

#Lädt Sample
sample_idx = 0
tensor, sr, _ = filedataset_ava[0]
tensor = tensor[0: librosa.time_to_samples(times=LENGTH_CHECK, sr=sr)]
tensor = torch.concat([torch.zeros(size=(CONTEXT_LENGTH,)), tensor])

In [9]:
model = models.silerio.get_model()

In [10]:
#Iterriert über Sample
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

times = []

with torch.no_grad():
    for sample in get_samples(tensor, SAMPLE_LENGTH + CONTEXT_LENGTH, HOP_LENGTH):
        
        start.record()
        y = model(sample, sr)
        end.record()
        
        torch.cuda.synchronize()
        times.append(start.elapsed_time(end))

#Output
np.average(times)

/mnt/data/source_jonas/VAD/Scripte/ENV/lib/python3.10/site-packages/torch/nn/modules/module.py:1501: UserWarning: operator() profile_node %669 : int[] = prim::profile_ivalue(%667)
 does not have profile information (Triggered internally at ../third_party/nvfuser/csrc/graph_fuser.cpp:104.)
  return forward_call(*args, **kwargs)


0.6451942056973775

In [5]:
model = models.STFTCNN.load_from_checkpoint(
    #Path
    checkpoint_path=r"lightning_logs/STFTCNN_CONTEXT_LENGTH-4096/version_2/checkpoints/last.ckpt",

    #Params
    stft_sr=SAMPLE_RATE
)
model.cpu()
model.eval()

STFT kernels created, time used = 0.0026 seconds


STFTCNN(
  (accuracy): BinaryAccuracy()
  (stft): STFT(n_fft=512, Fourier Kernel size=(257, 1, 512), iSTFT=False, trainable=False)
  (first_cnn_layer): Block2D(
    (conv): Conv2d(1, 32, kernel_size=(16, 16), stride=(1, 1), padding=same)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (block_list): ModuleList(
    (0): Block2D(
      (conv): Conv2d(32, 32, kernel_size=(16, 16), stride=(1, 1), padding=same)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (last_cnn_layer): Block2D(
    (conv): Conv2d(32, 32, kernel_size=(16, 16), stride=(1, 1), padding=same)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (fc1): Linear(in_features=32, out_features=32, bias=True)
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=32, out_features=1, bias=True)
)

In [6]:
model = models.MELCNN.load_from_checkpoint(
    #Path
    checkpoint_path=r"lightning_logs/MELCNN_CONTEXT_LENGTH-3584/version_0/checkpoints/last.ckpt",

    #Params
    sr = SAMPLE_RATE,
    fft_window = torch.hann_window(4096)
)
model.cpu()
model.eval()

MELCNN(
  (accuracy): BinaryAccuracy()
  (fft): FFT()
  (mel_transformer): MelScale()
  (first_cnn_layer): Block(
    (conv): MyConv1dPadSame(
      (conv): Conv1d(1, 32, kernel_size=(16,), stride=(1,))
    )
  )
  (block_list): ModuleList(
    (0): Block(
      (conv): MyConv1dPadSame(
        (conv): Conv1d(32, 32, kernel_size=(16,), stride=(1,))
      )
      (bn): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (last_cnn_layer): Block(
    (conv): MyConv1dPadSame(
      (conv): Conv1d(32, 32, kernel_size=(16,), stride=(1,))
    )
    (bn): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (fc1): Linear(in_features=32, out_features=32, bias=True)
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=32, out_features=1, bias=True)
)

In [8]:
model(get_samples(tensor, SAMPLE_LENGTH + CONTEXT_LENGTH, HOP_LENGTH)[0])

RuntimeError: running_mean should contain 64 elements not 32

In [9]:
#Iterriert über Sample
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

times = []

with torch.no_grad():
    for sample in get_samples(tensor, SAMPLE_LENGTH + CONTEXT_LENGTH, HOP_LENGTH):
        
        start.record()
        y = model(sample.unsqueeze(0))
        end.record()
        
        torch.cuda.synchronize()
        times.append(start.elapsed_time(end))

#Output
np.average(times)